In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

# 데이터 읽기
file_path = '/content/train.csv'  # 파일 경로를 적절히 변경하세요
data = pd.read_csv(file_path)

# 결측값 처리 (여기서는 간단히 'WT'로 대체)
data.fillna('WT', inplace=True)

# 특징과 타겟 분리
X = data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y = data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 1로 변환
X_encoded = X.apply(lambda col: col.map(lambda x: 0 if x == 'WT' else 1))

# 레이블 인코딩
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# 모델 구성
model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),  # 드롭아웃 추가
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),  # 드롭아웃 추가
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정
])

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=16)  # 에포크 수와 배치 크기 조정

# 모델 평가
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
272/272 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.1436 - loss: 3.0985 - val_accuracy: 0.2631 - val_loss: 2.4670
Epoch 2/100
272/272 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.3701 - loss: 2.0850 - val_accuracy: 0.3054 - val_loss: 2.1753
Epoch 3/100
272/272 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.5464 - loss: 1.3660 - val_accuracy: 0.3192 - val_loss: 2.3670
Epoch 4/100
272/272 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.6900 - loss: 0.9085 - val_accuracy: 0.3073 - val_loss: 2.6966
Epoch 5/100
272/272 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.7803 - loss: 0.6039 - val_accuracy: 0.2843 - val_loss: 2.9735
Epoch 6/100
272/272 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.8229 - loss: 0.4871 - val_accuracy: 0.2944 - val_loss: 3.2975
Epoch 7/100
272/272 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.8589 - loss: 0.4139 - val_accuracy: 0.2760 - val_loss: 3.4524
Epoch 8/100
272/272 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.8762 - loss: 0.3489 - 

In [2]:

# 테스트 데이터 읽기
test_file_path = '/content/test.csv'  # 테스트 데이터 파일 경로
test_data = pd.read_csv(test_file_path)

# 결측값 처리
test_data.fillna('WT', inplace=True)

# 특징 인코딩: 'WT'는 0, 그 외는 1로 변환
X_test_encoded = test_data.drop(columns=['ID']).apply(lambda col: col.map(lambda x: 0 if x == 'WT' else 1))

# 예측 수행
predictions = model.predict(X_test_encoded)

# 예측 결과 변환 (가장 높은 확률의 클래스를 선택)
predicted_classes = predictions.argmax(axis=1)

# 레이블 인코딩된 클래스를 원래 클래스 이름으로 변환
predicted_labels = le.inverse_transform(predicted_classes)

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('predictions.csv', index=False)

print("Predictions saved to 'predictions.csv'.")


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Predictions saved to 'predictions.csv'.


In [3]:
results['Predicted_SUBCLASS'].value_counts()

,count
Predicted_SUBCLASS,
STES,402
BRCA,265
KIPAN,199
SKCM,167
UCEC,165
GBMLGG,146
COAD,132
BLCA,97
HNSC,79


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# 학습 데이터 읽기
train_file_path = '/content/train.csv'  # 학습 데이터 파일 경로
train_data = pd.read_csv(train_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)

# 테스트 데이터 읽기
test_file_path = '/content/test.csv'  # 테스트 데이터 파일 경로
test_data = pd.read_csv(test_file_path)

# 결측값 처리
test_data.fillna('WT', inplace=True)

# 테스트 데이터에 SUBCLASS 열 추가 (NaN으로 설정)
test_data['SUBCLASS'] = None

# 학습 데이터와 테스트 데이터 결합
combined_data = pd.concat([train_data, test_data], ignore_index=True)

# 특징과 타겟 분리
X = combined_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y = combined_data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 1로 변환
X_encoded = X.apply(lambda col: col.map(lambda x: 0 if x == 'WT' else 1))

# 레이블 인코딩 (학습 데이터에만 적용)
le = LabelEncoder()
y_encoded = le.fit_transform(y.dropna())  # NaN을 제외하고 인코딩

# 학습 데이터와 테스트 데이터 분리
y_encoded_full = pd.Series([None] * len(combined_data), index=combined_data.index)  # 초기화
y_encoded_full.iloc[:len(y_encoded)] = y_encoded  # 학습 데이터에 대해서만 값 설정

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded[:len(y_encoded)], y_encoded, test_size=0.2, random_state=42)

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정
])

# 모델 컴파일
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# 모델 학습
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=16, callbacks=[early_stopping])

# 모델 평가

# 예측 수행 (테스트 데이터)
X_test_encoded = X_encoded[len(y_encoded):]  # 테스트 데이터 부분
predictions = model.predict(X_test_encoded)

# 예측 결과 변환 (가장 높은 확률의 클래스를 선택)
predicted_classes = predictions.argmax(axis=1)

# 레이블 인코딩된 클래스를 원래 클래스 이름으로 변환
predicted_labels = le.inverse_transform(predicted_classes)

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('predictions3.csv', index=False)

print("Predictions saved to 'predictions.csv'.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - accuracy: 0.1594 - loss: 2.9499 - val_accuracy: 0.3352 - val_loss: 2.1929
Epoch 2/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 37ms/step - accuracy: 0.5058 - loss: 1.5475 - val_accuracy: 0.3594 - val_loss: 2.1839
Epoch 3/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.7414 - loss: 0.7789 - val_accuracy: 0.3666 - val_loss: 2.5634
Epoch 4/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.8492 - loss: 0.4355 - val_accuracy: 0.3135 - val_loss: 3.1794
Epoch 5/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8765 - loss: 0.3408 - val_accuracy: 0.3280 - val_loss: 2.9928
Epoch 6/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.8950 - loss: 0.2307 - val_accuracy: 0.3207 - val_loss: 3.0950
Epoch 7/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.9036 - loss: 0.2018 - val_accuracy: 0.3102 - val_loss: 3.4553
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Predictions saved to 'predictions.csv'.


In [5]:
results

,ID,SUBCLASS
0,TEST_0000,KIPAN
1,TEST_0001,STES
2,TEST_0002,PCPG
3,TEST_0003,GBMLGG
4,TEST_0004,LUSC
...,...,...
2541,TEST_2541,SKCM
2542,TEST_2542,TGCT
2543,TEST_2543,UCEC
2544,TEST_2544,LAML


In [10]:
results.SUBCLASS.value_counts()

,count
SUBCLASS,
COAD,426
STES,352
BRCA,330
KIPAN,227
CESC,170
THCA,161
TGCT,143
GBMLGG,139
HNSC,130


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

# 학습 데이터 읽기
train_file_path = '/content/train.csv'  # 학습 데이터 파일 경로
train_data = pd.read_csv(train_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)

# 테스트 데이터 읽기
test_file_path = '/content/test.csv'  # 테스트 데이터 파일 경로
test_data = pd.read_csv(test_file_path)

# 결측값 처리
test_data.fillna('WT', inplace=True)

# 테스트 데이터에 SUBCLASS 열 추가 (NaN으로 설정)
test_data['SUBCLASS'] = None

# 학습 데이터와 테스트 데이터 결합
combined_data = pd.concat([train_data, test_data], ignore_index=True)

# 특징과 타겟 분리
X = combined_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y = combined_data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 1로 변환
X_encoded = X.apply(lambda col: col.map(lambda x: 0 if x == 'WT' else 1))

# 레이블 인코딩 (학습 데이터에만 적용)
le = LabelEncoder()
y_encoded = le.fit_transform(y.dropna())  # NaN을 제외하고 인코딩

# 학습 데이터와 테스트 데이터 분리
y_encoded_full = pd.Series([None] * len(combined_data), index=combined_data.index)  # 초기화
y_encoded_full.iloc[:len(y_encoded)] = y_encoded  # 학습 데이터에 대해서만 값 설정

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded[:len(y_encoded)], y_encoded, test_size=0.2, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정
])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=100, batch_size=16, callbacks=[early_stopping])

# 테스트 데이터 예측
X_test_encoded = X_encoded[len(y_encoded):]  # 테스트 데이터 부분
X_test_scaled = scaler.transform(X_test_encoded)  # 정규화 적용

predictions = model.predict(X_test_scaled)

# 예측 결과 변환 (가장 높은 확률의 클래스를 선택)
predicted_classes = predictions.argmax(axis=1)

# 레이블 인코딩된 클래스를 원래 클래스 이름으로 변환
predicted_labels = le.inverse_transform(predicted_classes)

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('predictions4.csv', index=False)

print("Predictions saved to 'predictions.csv'.")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 13s 34ms/step - accuracy: 0.1230 - loss: 3.5729 - val_accuracy: 0.2603 - val_loss: 2.7594
Epoch 2/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - accuracy: 0.3878 - loss: 2.2198 - val_accuracy: 0.2933 - val_loss: 2.4273
Epoch 3/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.5875 - loss: 1.3323 - val_accuracy: 0.3143 - val_loss: 2.6250
Epoch 4/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.7489 - loss: 0.7976 - val_accuracy: 0.3030 - val_loss: 2.7447
Epoch 5/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 19s 32ms/step - accuracy: 0.8074 - loss: 0.6120 - val_accuracy: 0.2699 - val_loss: 3.1657
Epoch 6/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 0.8341 - loss: 0.5509 - val_accuracy: 0.2836 - val_loss: 4.0648
Epoch 7/100
310/310 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.8188 - loss: 0.5413 - val_accuracy: 0.2925 - val_loss: 3.8485
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Predictions saved to 'predictions.csv'.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE  # SMOTE 사용을 위한 라이브러리 추가

# 학습 데이터 읽기
train_file_path = '/content/train.csv'  # 학습 데이터 파일 경로
train_data = pd.read_csv(train_file_path)

# 결측값 처리
train_data.fillna('WT', inplace=True)

# 테스트 데이터 읽기
test_file_path = '/content/test.csv'  # 테스트 데이터 파일 경로
test_data = pd.read_csv(test_file_path)

# 결측값 처리
test_data.fillna('WT', inplace=True)

# 테스트 데이터에 SUBCLASS 열 추가 (NaN으로 설정)
test_data['SUBCLASS'] = None

# 학습 데이터와 테스트 데이터 결합
combined_data = pd.concat([train_data, test_data], ignore_index=True)

# 특징과 타겟 분리
X = combined_data.drop(columns=['ID', 'SUBCLASS'])  # ID와 SUBCLASS 열 제거
y = combined_data['SUBCLASS']

# 특징 인코딩: 'WT'는 0, 그 외는 1로 변환
X_encoded = X.apply(lambda col: col.map(lambda x: 0 if x == 'WT' else 1))

# 레이블 인코딩 (학습 데이터에만 적용)
le = LabelEncoder()
y_encoded = le.fit_transform(y.dropna())  # NaN을 제외하고 인코딩

# 학습 데이터와 테스트 데이터 분리
y_encoded_full = pd.Series([None] * len(combined_data), index=combined_data.index)  # 초기화
y_encoded_full.iloc[:len(y_encoded)] = y_encoded  # 학습 데이터에 대해서만 값 설정

# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded[:len(y_encoded)], y_encoded, test_size=0.2, random_state=42)

# SMOTE로 불균형 클래스 처리
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 모델 구성
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train_resampled.shape[1],)),
    layers.Dropout(0.3),  # 드롭아웃 비율 조정
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),  # 드롭아웃 비율 조정
    layers.Dense(128, activation='relu'),
    layers.Dense(len(le.classes_), activation='softmax')  # 클래스 수에 맞추어 출력 노드 수 설정
])

# 모델 컴파일
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 조기 종료 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
model.fit(X_train_resampled, y_train_resampled, validation_data=(X_val, y_val), epochs=100, batch_size=16, callbacks=[early_stopping])

# 테스트 데이터 예측
X_test_encoded = X_encoded[len(y_encoded):]  # 테스트 데이터 부분

predictions = model.predict(X_test_encoded)

# 예측 결과 변환
predicted_classes = predictions.argmax(axis=1)

# 레이블 인코딩된 클래스를 원래 클래스 이름으로 변환
predicted_labels = le.inverse_transform(predicted_classes)

# 결과를 DataFrame으로 저장
results = pd.DataFrame({
    'ID': test_data['ID'],
    'SUBCLASS': predicted_labels
})

# 결과를 CSV 파일로 저장
results.to_csv('predictions_no_scaler.csv', index=False)

print("Predictions saved to 'predictions_no_scaler.csv'.")
